In [1]:
import os
import pandas as pd
import numpy as np
import re
import ast
from tqdm.auto import tqdm
from datetime import datetime, timedelta

from ib_async import *
import pandas_datareader.data as web
import wbgapi as wb
import country_converter as coco

from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

import gc
import argparse

from scipy.stats import skew
from scipy.optimize import minimize_scalar


In [2]:
# Data Cleaning
MAX_STALE_DAYS = 5
# Default params for detect_and_nullify_global_outliers
Z_THRESHOLD_GLOBAL_DEFAULT = 120.0
OUTLIER_WINDOW_DEFAULT = 5
# Params for detect_and_nullify_global_outliers in the main loop
Z_THRESHOLD_GLOBAL_LOOP = 50

# Walk-Forward Analysis
WALK_FORWARD_WINDOW_YEARS = range(3, 5)
TRAINING_PERIOD_DAYS = 365
MOMENTUM_PERIODS_DAYS = {
    '1y':  TRAINING_PERIOD_DAYS,
    '6mo': TRAINING_PERIOD_DAYS // 2,
    '3mo': TRAINING_PERIOD_DAYS // 4,
}

# Asset Filtering
MAX_GAP_LOG = 3.05
MAX_PCT_MISSING = 0.3

# Factor Construction
FACTOR_SCALING_FACTOR = 0.6

# Factor Screening
CORRELATION_THRESHOLD = 0.95

# Elastic Net Hyperparameters
ENET_ALPHAS = np.logspace(-11, -4, 30)
ENET_L1_RATIOS = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1]
ENET_CV = 5
ENET_TOL = 5e-4

In [3]:
def fetch_world_bank_data(all_country_codes, start_date, end_date, indicators):
    valid_country_codes = {code for code in all_country_codes if code is not None}
    try:
        wb_economies = {e['id'] for e in wb.economy.list()}
    except Exception as e:
        raise Exception(f"FATAL: Failed to fetch economy list from World Bank API: {e}")

    final_economies = sorted([code for code in valid_country_codes if code in wb_economies])
    unrecognized = valid_country_codes - set(final_economies)
    if unrecognized:
        print(f"Info: The following economies were not recognized by the World Bank API and will be skipped: {unrecognized}")
    if not final_economies:
        raise Exception("Error: No valid economies found to query the World Bank API.")

    all_data = []
    chunk_size = 40
    for i in range(0, len(final_economies), chunk_size):
        chunk = final_economies[i:i + chunk_size]
        try:
            data_chunk = wb.data.DataFrame(list(indicators), chunk, time=range(start_date.year - 5, end_date.year + 1), labels=False)
            all_data.append(data_chunk)
        except wb.APIError as e:
            print(f"API Error fetching data for chunk {i//chunk_size + 1}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred fetching data for chunk {i//chunk_size + 1}: {e}")

    if not all_data:
        raise Exception("Error: Failed to retrieve any data from the World Bank.")

    return pd.concat(all_data)

def evaluate_literal(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val
    
def load(path):
    df = pd.read_csv(path)
    for col in df.columns:
        df[col] = df[col].apply(evaluate_literal)
    return df

def verify_files(verified_path, data_path):
    try:
        return pd.read_csv(verified_path)
    except FileNotFoundError:
        util.startLoop()
        ib = IB()
        ib.connect('127.0.0.1', 7497, clientId=2)

        file_list = os.listdir(data_path)
        verified_files = []

        for file_name in tqdm(file_list, total=len(file_list), desc="Verifying files"):
            if not file_name.endswith('.csv'):
                continue
            try:
                symbol, exchange, currency = file_name.replace('.csv', '').split('-')
                symbol_data = fund_df[(fund_df['symbol'] == symbol) & (fund_df['currency'] == currency)]
                if symbol_data.empty:
                    continue

                contract_details = ib.reqContractDetails(Stock(symbol, exchange, currency))
                if not contract_details:
                    continue
                isin = contract_details[0].secIdList[0].value

                if symbol_data['isin'].iloc[0] != isin:
                    continue

                instrument_name = symbol_data['longName'].iloc[0].replace('-', '').replace('+', '')
                leveraged = any(
                    re.fullmatch(r'\d+X', word) and int(word[:-1]) > 1 or word.lower().startswith(('lv', 'lev'))
                    for word in instrument_name.split()
                )
                if leveraged:
                    continue

                verified_files.append({'symbol': symbol, 'currency': currency})
            except ValueError as e:
                print(f"Invalid filename format {file_name}: {e}")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

        verified_df = pd.DataFrame(verified_files)
        verified_df.to_csv(verified_path, index=False)

        ib.disconnect()
        util.stopLoop()
        
        return verified_df

def ensure_series_types(df, price_col):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    for col in ['volume', price_col]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def validate_raw_prices(df, price_col):
    invalid_price_mask = df[price_col] <= 0
    inconsistent_mask = pd.Series(False, index=df.index)
    if 'low' in df.columns and 'high' in df.columns:
        inconsistent_mask = (df['low'] > df['high'])
    local_error_mask = invalid_price_mask | inconsistent_mask
    df = df[~local_error_mask].copy()
    return df

def handle_stale_periods(df, price_col, max_stale_days=MAX_STALE_DAYS):
    stale_groups = (df[price_col].diff() != 0).cumsum()
    if stale_groups.empty:
        return df
    period_lengths = df.groupby(stale_groups)[price_col].transform('size')
    long_stale_mask = period_lengths > max_stale_days
    is_intermediate_stale_row = (stale_groups.duplicated(keep='first') & stale_groups.duplicated(keep='last'))
    rows_to_drop_mask = long_stale_mask & is_intermediate_stale_row
    df = df[~rows_to_drop_mask].copy()
    return df

def detect_and_nullify_global_outliers(meta_df, price_col, z_threshold=Z_THRESHOLD_GLOBAL_DEFAULT, window=OUTLIER_WINDOW_DEFAULT):
    all_pct_changes = pd.concat(
        [row['df']['pct_change'] for _, row in meta_df.iterrows()],
        ignore_index=True
    ).dropna()
    all_pct_changes = all_pct_changes[~np.isinf(all_pct_changes) & (all_pct_changes != 0)]

    global_median_return = all_pct_changes.median()
    global_mad = (all_pct_changes - global_median_return).abs().median()

    for idx, row in meta_df.iterrows():
        df = row['df']
        df = df.reset_index(drop=True)
        if df['pct_change'].isnull().all():
            continue
        cols_to_null = [price_col, 'volume', 'high', 'low', 'pct_change']
        cols_to_null = [c for c in cols_to_null if c in df.columns]

        absolute_modified_z = (df['pct_change'] - global_median_return).abs() / global_mad
        outlier_mask = absolute_modified_z > z_threshold

        if outlier_mask.any():

            candidate_indices = df.index[outlier_mask]
            for df_idx in candidate_indices:
                price_to_check_idx = df_idx - 1
                price_to_check = df.loc[price_to_check_idx, price_col]
                local_window_start = max(0, price_to_check_idx - window)
                local_window = df.loc[local_window_start : price_to_check_idx - 1, price_col].dropna()
                local_mean = local_window.mean()
                local_std = local_window.std()
                if local_std != 0: 
                    price_z_score = abs(price_to_check - local_mean) / local_std
                    if price_z_score > z_threshold / 10:
                        df.loc[price_to_check_idx, cols_to_null] = np.nan

                price_to_check = df.loc[df_idx, price_col]
                local_window_end = min(df_idx + window, df.index[outlier_mask].max())
                local_window = df.loc[df_idx + 1: local_window_end, price_col].dropna()
                local_mean = local_window.mean()
                local_std = local_window.std()
                if local_std != 0:
                    price_z_score = abs(price_to_check - local_mean) / local_std
                    if price_z_score > z_threshold / 10:
                        df.loc[df_idx, cols_to_null] = np.nan

            df['pct_change'] = df[price_col].pct_change(fill_method=None)
            meta_df.at[idx, 'df'] = df

def calculate_slope(value1, value2, time1, time2):
    return (value1 - value2) / (time1 - time2)

def get_return_stats(df, training_cutoff, momentum_cutoffs, risk_free_df):
    training_df = df[df.index < training_cutoff]
    training_rf = risk_free_df[risk_free_df.index < training_cutoff]

    excess_returns = training_df['pct_change'] - training_rf['daily_nominal_rate']
    sharpe = excess_returns.mean() / excess_returns.std()

    momentum_3mo = training_df[training_df.index >= momentum_cutoffs['3mo']]['pct_change'].mean()
    momentum_6mo = training_df[training_df.index >= momentum_cutoffs['6mo']]['pct_change'].mean()
    momentum_1y  = training_df[training_df.index >= momentum_cutoffs['1y']]['pct_change'].mean()

    return pd.Series(
        [momentum_3mo, momentum_6mo, momentum_1y, sharpe],
        index=['momentum_3mo', 'momentum_6mo', 'momentum_1y', 'stats_sharpe']
    )

def create_continent_map(standard_names):
    continents = cc.convert(names=standard_names, to='continent', not_found=None)
    return {name: (cont if cont is not None else 'Other')
            for name, cont in zip(standard_names, continents)}

def calculate_country_stats(wb_data_full, standard_names, end_year, window_size=3):
    countries_in_window = [name for name in standard_names if name in wb_data_full.index.get_level_values('economy')]
    if not countries_in_window:
        return pd.DataFrame()
    
    data = wb_data_full.loc[countries_in_window].dropna(axis=1)
    available_years = [int(col.replace('YR', '')) for col in data.columns]

    cols_to_keep = [col for col, year in zip(data.columns, available_years) if year <= end_year.year]
    data = data[cols_to_keep].copy()
    data.dropna(axis=1, inplace=True);

    yoy_change = data.diff(axis=1)
    first_div = yoy_change.T.rolling(window=window_size).mean().T

    yoy_change_first_div = first_div.diff(axis=1)
    second_div = yoy_change_first_div.T.rolling(window=window_size).mean().T

    latest_year_col = data.columns[-1]
    latest_first_div_col = first_div.columns[-1]
    latest_second_div_col = second_div.columns[-1]

    derivatives = pd.DataFrame(data[latest_year_col])
    derivatives.rename(columns={latest_year_col: 'raw_value'}, inplace=True)
    derivatives['1st_div'] = first_div[latest_first_div_col] / derivatives['raw_value']
    derivatives['2nd_div'] = second_div[latest_second_div_col] / derivatives['raw_value']
    
    metric_df_reshaped = derivatives.unstack(level='series')
    if isinstance(metric_df_reshaped.columns, pd.MultiIndex):
         metric_df_final = metric_df_reshaped.swaplevel(0, 1, axis=1)
         metric_df_final.sort_index(axis=1, level=0, inplace=True)
    else:
         metric_df_final = metric_df_reshaped

    return metric_df_final

def construct_long_short_factor_returns(full_meta_df, returns_df, long_symbols, short_symbols, factor_column=None):
    long_df = full_meta_df[full_meta_df['conId'].isin(long_symbols)].set_index('conId')
    long_weights = long_df['profile_cap_usd'].reindex(returns_df.columns).fillna(0)
    if factor_column:
        factor_weights = (full_meta_df[factor_column].max() - long_df[factor_column]) / (full_meta_df[factor_column].max() - full_meta_df[factor_column].min())
        factor_weights = factor_weights.reindex(returns_df.columns).fillna(0)
        if factor_weights.sum() != 0:
            long_weights *= factor_weights

    if long_weights.sum() != 0:
        long_weights /= long_weights.sum()
    long_returns = returns_df.dot(long_weights)
    
    short_df = full_meta_df[full_meta_df['conId'].isin(short_symbols)].set_index('conId')
    short_weights = short_df['profile_cap_usd'].reindex(returns_df.columns).fillna(0)
    if factor_column:
        factor_weights = (short_df[factor_column] - full_meta_df[factor_column].min()) / (full_meta_df[factor_column].max() - full_meta_df[factor_column].min())
        factor_weights = factor_weights.reindex(returns_df.columns).fillna(0)
        if factor_weights.sum() != 0:
            short_weights *= factor_weights

    if short_weights.sum() != 0:
        short_weights /= short_weights.sum()
    short_returns = returns_df.dot(short_weights)
    
    factor_returns = long_returns - short_returns
    return factor_returns

def construct_factors(filtered_df, pct_changes, portfolio_dfs, risk_free_df, scaling_factor=FACTOR_SCALING_FACTOR):
    factors = {}
    # Market risk premium
    factors['factor_market_premium'] = (portfolio_dfs['equity']['pct_change'] - risk_free_df['daily_nominal_rate'])

    # SMB_ETF
    small_symbols = filtered_df[filtered_df['marketcap_small'] == 1]['conId'].tolist()
    large_symbols = filtered_df[filtered_df['marketcap_large'] == 1]['conId'].tolist()

    intersection = set(small_symbols) & set(large_symbols)
    small_symbols = [s for s in small_symbols if s not in intersection]
    large_symbols = [s for s in large_symbols if s not in intersection]
    smb_etf = construct_long_short_factor_returns(filtered_df, pct_changes, small_symbols, large_symbols)
    factors['factor_smb'] = smb_etf

    # # HML_ETF
    # value_cols = [col for col in filtered_df.columns if col.startswith('style_') and col.endswith('value')]
    # growth_cols = [col for col in filtered_df.columns if col.startswith('style_') and col.endswith('growth')]
    # value_symbols = filtered_df[filtered_df[value_cols].ne(0).any(axis=1)]['conId'].tolist()
    # growth_symbols = filtered_df[filtered_df[growth_cols].ne(0).any(axis=1)]['conId'].tolist()

    # intersection = set(value_symbols) & set(growth_symbols)
    # value_symbols = [s for s in value_symbols if s not in intersection]
    # growth_symbols = [s for s in growth_symbols if s not in intersection]
    # hml_etf = construct_long_short_factor_returns(filtered_df, pct_changes, value_symbols, growth_symbols)
    # factors['factor_hml'] = hml_etf

    # Metadata
    excluded = ['style_', 'marketcap_', 'countries_', 'fundamentals_', 'momentum_']
    numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]
    for col in numerical_cols:
        if not any(col.startswith(prefix) for prefix in excluded):
            try:
                std = filtered_df[col].std()
                mean = filtered_df[col].mean()

                upper_boundary = min(filtered_df[col].max(), mean + (scaling_factor * std))
                lower_boundary = max(filtered_df[col].min(), mean - (scaling_factor * std))

                low_factor_symbols = filtered_df[filtered_df[col] <= lower_boundary]['conId'].tolist()
                high_factor_symbols = filtered_df[filtered_df[col] >= upper_boundary]['conId'].tolist()
                if col.endswith('variety'):
                    var_etf = construct_long_short_factor_returns(filtered_df, pct_changes, low_factor_symbols, high_factor_symbols, factor_column=col)
                else:
                    var_etf = construct_long_short_factor_returns(filtered_df, pct_changes, high_factor_symbols, low_factor_symbols, factor_column=col)
                factors[col] = var_etf

            except Exception as e:
                print(col)
                print(e)
                raise

    return pd.DataFrame(factors)

def prescreen_factors(factors_df, correlation_threshold=CORRELATION_THRESHOLD, drop_map=None):
    if factors_df is None or factors_df.empty or factors_df.shape[1] == 0:
        raise ValueError("factors_df must be a non-empty DataFrame with at least one column.")
    temp_factors_df = factors_df.copy()

    corr_matrix = temp_factors_df.corr().abs()
    corr_pairs = corr_matrix.where(np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)).stack()
    corr_pairs = corr_pairs.sort_values(ascending=False)

    if not drop_map:
        drop_map = {}
    col_order = list(temp_factors_df.columns)
    for (col1, col2), corr_val in corr_pairs.items():
        if corr_val < correlation_threshold:
            break

        already_dropped = {c for drops in drop_map.values() for c in drops}
        if col1 in already_dropped or col2 in already_dropped:
            continue

        if col_order.index(col1) < col_order.index(col2):
            keeper, to_drop = col1, col2
        else:
            keeper, to_drop = col2, col1

        drop_map.setdefault(keeper, []).append(to_drop)

    cols_to_drop = set(col for drops in drop_map.values() for col in drops)
    temp_factors_df = temp_factors_df.drop(columns=cols_to_drop)
    return temp_factors_df, drop_map

def merge_drop_map(drop_map):
    cols_to_drop = set(col for drops in drop_map.values() for col in drops)
    final_drop_map = {}
    for keeper, direct_drops in drop_map.items():
        if keeper not in cols_to_drop:
            cols_to_check = list(direct_drops) 
            all_related_drops = set(direct_drops)
            while cols_to_check:
                col = cols_to_check.pop(0)
                if col in drop_map:
                    new_drops = [d for d in drop_map[col] if d not in all_related_drops]
                    cols_to_check.extend(new_drops)
                    all_related_drops.update(new_drops)
            
            final_drop_map[keeper] = sorted(list(all_related_drops))
    
    return final_drop_map

def run_regressions(distilled_factors):
    results = []
    for symbol in pct_changes.columns:
        etf_excess = pct_changes[symbol] - risk_free_df['daily_nominal_rate']
        data = pd.concat([etf_excess.rename('etf_excess'), distilled_factors], axis=1).dropna()

        Y = data['etf_excess']
        X = sm.add_constant(data.iloc[:, 1:])
        model = sm.OLS(Y, X).fit()
        result = {
            'conId': symbol,
            'nobs': model.nobs,
            'r_squared': model.rsquared,
            'r_squared_adj': model.rsquared_adj,
            'f_statistic': model.fvalue,
            'f_pvalue': model.f_pvalue,
            'aic': model.aic,
            'bic': model.bic,
            'condition_number': model.condition_number,
            'alpha': model.params['const'],
            'alpha_pval': model.pvalues['const'],
            'alpha_tval': model.tvalues['const'],
            'alpha_bse': model.bse['const'],
        }
        for factor in distilled_factors.columns:
            result[f'beta_{factor}'] = model.params[factor]
            result[f'pval_beta_{factor}'] = model.pvalues[factor]
            result[f'tval_beta_{factor}'] = model.tvalues[factor]
            result[f'bse_beta_{factor}'] = model.bse[factor]
        results.append(result)

    results_df = pd.DataFrame(results)
    return results_df

def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data.sort_values(by='VIF', ascending=False)

def run_elastic_net(
                    factors_df,
                    pct_changes,
                    risk_free_df,
                    training_cutoff,
                    alphas=ENET_ALPHAS,
                    l1_ratio=ENET_L1_RATIOS,
                    cv=ENET_CV,
                    tol=ENET_TOL,
                    random_state=42):

    data = data = (
        factors_df.copy()
        .join(pct_changes, how='inner')
        .join(risk_free_df[['daily_nominal_rate']], how='inner')
        .fillna(0)
    )

    train = data[data.index < training_cutoff]
    test = data[data.index >= training_cutoff]

    X_train = train[factors_df.columns].values
    X_test = test[factors_df.columns].values
    
    metrics = []
    for etf in tqdm(pct_changes.columns, total=len(pct_changes.columns), desc="Elastic Net Regression"):
        Y_train = train[etf].values - train['daily_nominal_rate'].values
        Y_test = test[etf].values - test['daily_nominal_rate'].values

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('enet', ElasticNetCV(alphas=alphas,
                                l1_ratio=l1_ratio,
                                cv=cv,
                                random_state=random_state,
                                max_iter=499999,
                                tol=tol,
                                fit_intercept=True,
                                n_jobs=-1)),
        ])

        try:
            pipeline.fit(X_train, Y_train)
        except ValueError as e:
            print(f"Skipping {etf} due to error: {e}")
            continue

        # Unscale coefficients and intercept
        enet = pipeline.named_steps['enet']
        scaler = pipeline.named_steps['scaler']
        betas_train = enet.coef_ / scaler.scale_
        intercept = enet.intercept_ - np.dot(betas_train, scaler.mean_)

        # out-of-sample stats
        er_test = pipeline.predict(X_test)

        # in-sample stats
        er_train = pipeline.predict(X_train)

        row = {
            'conId': etf,
            'jensens_alpha': intercept,
            'enet_alpha': enet.alpha_,
            'l1_ratio': enet.l1_ratio_,
            'n_iter': enet.n_iter_,
            'dual_gap': enet.dual_gap_,
            'n_nonzero': np.sum(np.abs(betas_train) > 1e-6),
            'cv_mse_best': np.min(enet.mse_path_.mean(axis=2)),
            'cv_mse_average': np.mean(enet.mse_path_.mean(axis=2)),
            'cv_mse_worst': np.max(enet.mse_path_.mean(axis=2)),
            'mse_test' : mean_squared_error(Y_test, er_test),
            'mse_train' : mean_squared_error(Y_train, er_train),
            'r2_test' : r2_score(Y_test, er_test),
            'r2_train' : r2_score(Y_train, er_train),
        }

        # Map back coefficients to factor names
        for coef, fname in zip(betas_train, factors_df.columns):
            row[f'{fname}_beta'] = coef

        metrics.append(row)
    
    results_df = pd.DataFrame(metrics).set_index('conId')
    return results_df

def optimize_scalar(series):
    def obj(s):
        if s <= 0:
            return np.inf
        return skew(np.log1p(s * series))**2

    result = minimize_scalar(obj, bounds=(1e-5, 1e20), method='bounded')
    print(result.x)
    return result.x

In [4]:
kind = 'trades'
root = 'data/daily-trades/'
data_path = root + 'series/'
verified_path = root + 'verified_files.csv'
price_col = 'average'

fund_df = load('data/fundamentals.csv')
fund_df['funds_date'] = pd.to_datetime(fund_df['funds_date'])
verified_df = verify_files(verified_path, data_path)

In [5]:
# Load full historical price series
if 'meta' not in globals():# or input('Reload CSVs? (y/n)').lower().strip() == 'y':
    last_date = (datetime.now() - timedelta(days=365 * 99))
    first_date = (datetime.now())
    meta = []
    file_list = os.listdir(data_path)
    for file in tqdm(file_list, total=len(file_list), desc="Loading files"):
        if not file.endswith('.csv'):
            continue
        parts = os.path.splitext(file)[0].split('-')
        symbol, exchange, currency = parts[0], parts[1], parts[2]
        if not ((verified_df['symbol'] == symbol) & (verified_df['currency'] == currency)).any():
            continue
        try:
            df = load(data_path + file)
            df = ensure_series_types(df, price_col)
            df = validate_raw_prices(df, price_col)
            df = handle_stale_periods(df, price_col)
            df['pct_change'] = df[price_col].pct_change()

            if df['date'].max() > last_date:
                last_date = df['date'].max()
            if df['date'].min() < first_date:
                first_date = df['date'].min()
            
            meta.append({
                'symbol': symbol,
                'currency': currency,
                'exchange_api': exchange,
                'df': df[['date', price_col, 'volume', 'pct_change']],
            })
        except Exception as e:
            raise Exception(f"ERROR loading {file}: {e}")

    meta = pd.DataFrame(meta)
    detect_and_nullify_global_outliers(meta, price_col=price_col, z_threshold=Z_THRESHOLD_GLOBAL_LOOP, window=OUTLIER_WINDOW_DEFAULT)

Loading files:   0%|          | 0/5330 [00:00<?, ?it/s]

In [6]:
# Download supplementary data
if 'risk_free_df_full' not in globals():# or input('Redownload supplementary data? (y/n)').lower().strip() == 'y':
    # Risk-free series calculation
    tickers = {
        'US': 'DTB3',
        'Canada': 'IR3TIB01CAM156N',
        'Germany': 'IR3TIB01DEM156N',
        'UK': 'IR3TIB01GBM156N',
        'France': 'IR3TIB01FRA156N',
    }
    bonds = {}
    failed = []
    for country, ticker in tickers.items():
        try:
            series = web.DataReader(ticker, 'fred', first_date, last_date)
            bonds[country] = series / 100.0
        except Exception:
            try:
                series = web.DataReader(ticker, 'oecd', first_date, last_date)
                bonds[country] = series / 100.0
            except Exception as oecd_err:
                failed.append(country)

    # Combine into a single DataFrame
    df_bonds = pd.concat(bonds, axis=1)
    df_bonds.columns = [c for c in tickers if c not in failed]
    df_bonds = df_bonds.interpolate(method='akima').bfill().ffill()

    risk_free_df_full = df_bonds.mean(axis=1).rename('nominal_rate')
    business_days = pd.date_range(start=first_date, end=last_date, freq='B')
    risk_free_df_full = risk_free_df_full.reindex(business_days, copy=False)

    risk_free_df_full = pd.DataFrame(risk_free_df_full)
    risk_free_df_full['daily_nominal_rate'] = risk_free_df_full['nominal_rate'] / 252

    # Get country stats
    indicator_name_map = {
        'NY.GDP.PCAP.CD': 'gdp_pcap',
        'SP.POP.TOTL': 'population',
    }
    cc = coco.CountryConverter()

    all_country_cols = [col for col in fund_df.columns if col.startswith('countries') and not col.endswith('variety')]
    all_possible_standard_names = set()
    for col in all_country_cols:
        raw_name = col.replace('countries_', '').replace(' ', '')
        standard_name = cc.convert(names=raw_name, to='ISO3', not_found=None)
        if standard_name:
            all_possible_standard_names.add(standard_name)

    world_bank_data_full = fetch_world_bank_data(all_possible_standard_names, first_date, last_date, indicator_name_map.keys())

Unidentified not found in regex


Info: The following economies were not recognized by the World Bank API and will be skipped: {'TWN', 'JEY', 'Unidentified'}


In [7]:
YEAR_RANGE = 5
training_oldest = last_date - timedelta(days=365 * YEAR_RANGE)

meta_window = meta.copy()
meta_window['df'] = meta['df'].apply(lambda df: df.loc[df['date'].between(training_oldest, last_date)].copy())
business_days = pd.date_range(start=training_oldest, end=last_date, freq='B')

for idx, row in meta_window.iterrows():
    df = row['df']
    merged = pd.DataFrame({'date': business_days}).merge(df, on='date', how='left')
    present = merged[price_col].notna()
    present_idx = np.flatnonzero(present)
    gaps = []
    length = len(merged)
    if present_idx.size > 0:
        if present_idx[0] > 0:
            gaps.append(present_idx[0])
        if present_idx.size > 1:
            internal_gaps = np.diff(present_idx) - 1
            gaps.extend(gap for gap in internal_gaps if gap > 0)
        if present_idx[-1] < length - 1:
            gaps.append(length - 1 - present_idx[-1])
    else:
        gaps = [length]
    gaps = np.array(gaps, dtype=int)
    gaps = gaps[gaps > 0]
    max_gap = float(gaps.max()) if gaps.size > 0 else 0.0
    std_gap = float(gaps.std()) if gaps.size > 0 else 0.0
    missing = length - present.sum()
    pct_missing = missing / length
    meta_window.at[idx, 'df'] = merged
    meta_window.at[idx, 'max_gap'] = max_gap
    meta_window.at[idx, 'missing'] = missing
    meta_window.at[idx, 'pct_missing'] = pct_missing
meta_window['max_gap_log'] = np.log1p(meta_window['max_gap'])

## static 3y window mean stats
condition = ((meta_window['max_gap_log'] < MAX_GAP_LOG) & (meta_window['pct_missing'] < MAX_PCT_MISSING))
filtered = meta_window[condition].copy()
print(f'{len(filtered)} ETFs included')
print(f'{len(meta_window) - len(filtered)} dropped')
del meta_window

for idx, row in filtered.iterrows():
    df = row['df']
    df[price_col] = df[price_col].interpolate(method='akima', limit_direction='both')
    if df[price_col].isna().any():
        df[price_col] = df[price_col].ffill()
        df[price_col] = df[price_col].bfill()
    df['pct_change'] = df[price_col].pct_change()
    filtered.at[idx, 'df'] = df.set_index('date')

# Isolate to one fundamental row per conId
training_cutoff = last_date - pd.Timedelta(days=TRAINING_PERIOD_DAYS)

before_training_end = fund_df[fund_df['funds_date'] <= training_cutoff]
if not before_training_end.empty:
    before_training_end = before_training_end.loc[before_training_end.groupby('conId')['funds_date'].idxmax()]
else:
    before_training_end = pd.DataFrame(columns=fund_df.columns)

after_training_end = fund_df[fund_df['funds_date'] > training_cutoff]
if not after_training_end.empty:
    after_training_end = after_training_end.loc[after_training_end.groupby('conId')['funds_date'].idxmin()]
else:
    after_training_end = pd.DataFrame(columns=fund_df.columns)

if not before_training_end.empty and not after_training_end.empty:
    after_training_end = after_training_end[~after_training_end['conId'].isin(before_training_end['conId'])]
spliced_fund_df = pd.concat([before_training_end, after_training_end])

filtered = pd.merge(filtered, spliced_fund_df, on=['symbol', 'currency'], how='inner').drop(['max_gap', 'missing', 'pct_missing', 'max_gap_log'], axis=1)
numerical_cols = [col for col in filtered.columns if filtered[col].dtype in [np.int64, np.float64] and col not in ['conId']]
pct_changes = pd.concat(
        [row['df']['pct_change'].rename(row['conId']) 
        for _, row in filtered.iterrows()], axis=1
    )

# Remove uninformative cols for market portfolios 
uninformative_cols = [col for col in numerical_cols if filtered[col].nunique(dropna=True) <= 1]
filtered = filtered.drop(columns=uninformative_cols)
filtered = filtered.dropna(axis=1, how='all')

# Add rate of change fundamentals
rate_fundamentals = [('EPSGrowth-1yr', 'EPS_growth_3yr', 'EPS_growth_5yr'),
                    ('ReturnonAssets1Yr', 'ReturnonAssets3Yr'),
                    ('ReturnonCapital', 'ReturnonCapital3Yr'),
                    ('ReturnonEquity1Yr', 'ReturnonEquity3Yr'),
                    ('ReturnonInvestment1Yr', 'ReturnonInvestment3Yr')]

for cols in rate_fundamentals:
    base_name = cols[0].replace('-1yr', '').replace('1Yr', '')
    slope_col = f'fundamentals_{base_name}_slope'
    if len(cols) == 3:
        col_1yr, col_3yr, col_5yr = cols
        filtered[slope_col] = calculate_slope(filtered[f'fundamentals_{col_1yr}'], filtered[f'fundamentals_{col_5yr}'], 1, 5)
        slope_1yr_3yr = calculate_slope(filtered[f'fundamentals_{col_1yr}'], filtered[f'fundamentals_{col_3yr}'], 1, 3)
        slope_3yr_5yr = calculate_slope(filtered[f'fundamentals_{col_3yr}'], filtered[f'fundamentals_{col_5yr}'], 3, 5)
        filtered[f'fundamentals_{base_name}_second_deriv'] = calculate_slope(slope_1yr_3yr, slope_3yr_5yr, 1, 3)
    elif len(cols) == 2:
        col_1yr, col_3yr = cols
        filtered[slope_col] = calculate_slope(filtered[f'fundamentals_{col_1yr}'], filtered[f'fundamentals_{col_3yr}'], 1, 3)
numerical_cols = [col for col in filtered.columns if filtered[col].dtype in [np.int64, np.float64] and col not in ['conId']]

# Return stats and split training and tests sets
momentum_cutoffs = {
    '1y':  training_cutoff - pd.Timedelta(days=MOMENTUM_PERIODS_DAYS['1y']),
    '6mo': training_cutoff - pd.Timedelta(days=MOMENTUM_PERIODS_DAYS['6mo']),
    '3mo': training_cutoff - pd.Timedelta(days=MOMENTUM_PERIODS_DAYS['3mo']),
}
risk_free_df = risk_free_df_full.loc[business_days]
filtered[['momentum_3mo', 'momentum_6mo', 'momentum_1y', 'stats_sharpe']] = filtered['df'].apply(lambda df: get_return_stats(df, training_cutoff, momentum_cutoffs, risk_free_df))

holding_cols = [col for col in filtered.columns if col.startswith('holding_') and col != 'holding_types_variety'] + ['total']
portfolio_dfs = {}

for holding_col in holding_cols:
    name = holding_col.split('_')[-1]
    if holding_col == 'total':
        weight = filtered['profile_cap_usd']
    else:
        weight = (filtered['profile_cap_usd'] * filtered[holding_col])

    total_market_cap = (weight).sum()
    filtered['weight'] = weight / total_market_cap
    
    weights = filtered.set_index('conId')['weight']
    portfolio_return = pct_changes.dot(weights)
    initial_price = 1
    portfolio_price = initial_price * (1 + portfolio_return.fillna(0)).cumprod()

    portfolio_df = pd.DataFrame({
        'date': portfolio_price.index,
        price_col: portfolio_price.values,
        'pct_change': portfolio_return.values
    }).set_index('date')

    portfolio_dfs[name] = portfolio_df

filtered.drop('weight', axis=1, inplace=True)

# Avoid dummy trap
empty_subcategories = {
'holding_types': ['other'],
'countries': ['Unidentified'], 
'currencies': ['<NoCurrency>'],
'industries': ['NonClassifiedEquity', 'NotClassified-NonEquity'],
'top10': ['OtherAssets', 'AccountsPayable','AccountsReceivable','AccountsReceivable&Pay','AdministrationFees','CustodyFees','ManagementFees','OtherAssetsandLiabilities','OtherAssetslessLiabilities', 'OtherFees','OtherLiabilities','Tax','Tax--ManagementFees'],
'debtors': ['OTHER'],
'maturity': ['%MaturityOther'],
'debt_type': ['%QualityNotAvailable', '%QualityNotRated'],
'manual': ['asset_other']
}

dummy_trap_cols = []
for k, lst in empty_subcategories.items():
    for i in lst:
        if k == 'manual':
            dummy_trap_cols.append(i)
        else:
            dummy_trap_cols.append(f'{k}_{i}')
    
filtered = filtered.drop(columns=dummy_trap_cols, axis=1, errors='ignore')
numerical_cols = [col for col in filtered.columns if filtered[col].dtype in [np.int64, np.float64] and col not in ['conId']]

# Select asset types to work on
asset_conditions = {
    'equity': (filtered['asset_equity'] == 1),
    'cash': (filtered['asset_cash'] == 1),
    'bond': (filtered['asset_bond'] == 1),
    'other': (filtered['asset_equity'] == 0) & (filtered['asset_cash'] == 0) & (filtered['asset_bond'] == 0),
}

exclude_assets = ['bond', 'cash']
asset_classes = list(asset_conditions.keys())

include_assets = [asset for asset in asset_classes if asset not in exclude_assets]
combined_condition = pd.Series(False, index=filtered.index)
for asset in include_assets:
    combined_condition |= asset_conditions[asset]

filtered_df = filtered[combined_condition]
numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]

single_value_columns = [col for col in filtered_df.columns if col in numerical_cols and filtered_df[col].nunique() == 1]
asset_cols = [col for col in filtered_df if col.startswith('asset')]
filtered_df = filtered_df.drop(columns=single_value_columns + asset_cols)
numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]

pct_changes = pct_changes[filtered_df['conId']]
del filtered

# Standardize country column names
cc = coco.CountryConverter()
country_cols = [col for col in filtered_df.columns if col.startswith('countries') and not col.endswith('variety')]
standard_names = set()
rename_map = {}
for col in country_cols:
    if col == 'countries_Unidentified':
        continue

    raw_name = col.replace('countries_', '').replace(' ', '')
    raw_name = ''.join([' ' + char if char.isupper() and i > 0 else char for i, char in enumerate(raw_name)]).strip()

    standard_name = cc.convert(names=raw_name, to='ISO3', not_found=None)
    standard_names.add(standard_name)
    if standard_name:
        rename_map[col] = f'countries_{standard_name}'
    else:
        print(f"Could not standardize: '{raw_name}' (from column '{col}')")
filtered_df.rename(columns=rename_map, inplace=True)

# Collapse country columns
metric_df = calculate_country_stats(world_bank_data_full, standard_names, last_date, window_size=3)
metric_suffixes = {
    'raw_value': '_value',
    '1st_div': '_growth',
    '2nd_div': '_acceleration'
}
for ind_code, ind_name in indicator_name_map.items():
    if ind_code in metric_df.columns.get_level_values(0):
        for metric_col, suffix in metric_suffixes.items():
            new_col_name = f'{ind_name}{suffix}'
            filtered_df[new_col_name] = 0.0

for std_name in standard_names:
    country_weight_col = f'countries_{std_name}'
    if country_weight_col not in filtered_df.columns:
        continue

    if std_name in metric_df.index:
        for ind_code, ind_name in indicator_name_map.items():
            if ind_code in metric_df.columns.get_level_values(0):
                for metric_col, suffix in metric_suffixes.items():
                    value = metric_df.loc[std_name, (ind_code, metric_col)]
                    target_col = f'{ind_name}{suffix}'
                    filtered_df[target_col] += filtered_df[country_weight_col] * value

# Drop single unique value columns
numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]
single_value_columns = [col for col in numerical_cols if filtered_df[col].nunique() == 1]
filtered_df = filtered_df.drop(columns=single_value_columns, errors='ignore')
numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]

fundamental_columns = [col for col in filtered_df.columns if col.startswith('fundamentals')]

value_columns_inverted = [
    'fundamentals_Price/Book',
    'fundamentals_Price/Cash',
    'fundamentals_Price/Earnings',
    'fundamentals_Price/Sales',
]
leverage_columns_inverted = [
    'fundamentals_LTDebt/Shareholders',
    'fundamentals_TotalDebt/TotalCapital',
    'fundamentals_TotalDebt/TotalEquity',
    'fundamentals_TotalAssets/TotalEquity',
]
profitability_columns = [
    'fundamentals_ReturnonAssets1Yr',
    'fundamentals_ReturnonAssets3Yr',
    'fundamentals_ReturnonCapital',
    'fundamentals_ReturnonCapital3Yr',
    'fundamentals_ReturnonEquity1Yr',
    'fundamentals_ReturnonEquity3Yr',
    'fundamentals_ReturnonInvestment1Yr',
    'fundamentals_ReturnonInvestment3Yr',
]
momentum_columns = [
    'momentum_3mo',
    'momentum_6mo',
    'momentum_1y',
    'fundamentals_RelativeStrength'
]
columns_to_scale = value_columns_inverted + leverage_columns_inverted + profitability_columns + momentum_columns
if any(x in filtered_df.columns for x in columns_to_scale):
    scaler = MinMaxScaler()
    filtered_df[columns_to_scale] = scaler.fit_transform(filtered_df[columns_to_scale])

    filtered_df['factor_value'] = (1 - filtered_df[value_columns_inverted]).sum(axis=1)
    filtered_df['factor_leverage'] = (1 - filtered_df[leverage_columns_inverted]).sum(axis=1)
    filtered_df['factor_profitability'] = filtered_df[profitability_columns].sum(axis=1)
    filtered_df['factor_momentum'] = filtered_df[momentum_columns].sum(axis=1)

    filtered_df = filtered_df.drop(columns=columns_to_scale, errors='ignore')

# Reorganize columns
categories = ['factor', 'holding_types', 'stats', 'momentum', 'profile', 'top10', 'population', 'msci', 'gdp', 'continent', 'countries', 'fundamentals', 'industries', 'currencies', 'debtors', 'maturity', 'debt_type', 'lipper', 'dividends', 'marketcap', 'style', 'domicile', 'asset']
numerical_cols = [col for col in filtered_df.columns if filtered_df[col].dtype in [np.int64, np.float64] and col not in ['conId']]
non_numerical = [col for col in filtered_df.columns if col not in numerical_cols]
for category in reversed(categories):
    cat_cols = [col for col in numerical_cols if col.startswith(category)]
    remaining = [col for col in numerical_cols if col not in cat_cols]
    numerical_cols = cat_cols + remaining
new_column_order = non_numerical + numerical_cols
filtered_df = filtered_df[new_column_order]

factors_df = construct_factors(filtered_df, pct_changes, portfolio_dfs, risk_free_df, scaling_factor=FACTOR_SCALING_FACTOR)

# Custom drop
low_absolute_beta = ['profile_cap_usd', 'holding_types_equity', 'industries_BasicMaterials', 'continent_Oceania_beta', 'holding_types_bond_beta']#, 'factor_smb_beta']
frequently_lassoed = ['gdp_pcap_growth', 'gdp_pcap_acceleration', 'continent_Africa', 'population_value', 'industries_Financials_beta', 'stats_sharpe']
walk_forward = ['industries_Healthcare_beta', 'continent_America_beta']#, 'factor_momentum_beta', 'factor_profitability_beta']
custom_drop = low_absolute_beta + frequently_lassoed + walk_forward
custom_drop = [c.split('_beta')[0] for c in custom_drop]
factors_df = factors_df.drop(columns=custom_drop, errors='ignore')

# Screen factors
distilled_factors, drop_map = prescreen_factors(factors_df, correlation_threshold=CORRELATION_THRESHOLD)
# corr_matrix = distilled_factors.corr()
# vif_df = calculate_vif(distilled_factors.dropna(axis=0))
# highest_vif = vif_df['VIF'].iloc[0]
# if distilled_factors.shape[1] > 2:
#     to_drop = vif_df['feature'].iloc[0]
#     distilled_factors.drop(columns=[to_drop], inplace=True)

# np.fill_diagonal(corr_matrix.values, 0)
# keeper = corr_matrix[to_drop].sort_values(ascending=False).index[0]
# drop_map.setdefault(keeper, []).append(to_drop)

del factors_df
calculate_vif(distilled_factors.dropna(axis=0))

2309 ETFs included
3021 dropped


,feature,VIF
4,factor_profitability,129.108366
8,top10_variety,68.933696
15,industries_Industrials,62.295322
17,industries_Technology,39.926269
11,gdp_pcap_value,37.192280
5,factor_momentum,35.996574
2,factor_value,35.802852
21,tradable,30.047162
20,industries_variety,27.445359
12,industries_ConsumerCyclicals,26.177291


In [8]:
# Print Final factors
drop_map = merge_drop_map(drop_map)
if drop_map:
    display(pd.Series(drop_map))
print(distilled_factors.shape)

categories = list(set([col.split('_')[0] for col in distilled_factors.columns]) | set(categories))
print('\n# Factors included:')
for cat in categories:
    if cat in ['tradable']:
        cat_list = ['tradable']
    else:
        cat_list = [col.split(cat)[-1].strip('_').capitalize() for col in distilled_factors.columns if col.startswith(cat)]
    if cat_list:
        print(f'{(',  ').join(cat_list)}')

(1304, 22)

# Factors included:
Pcap_value
Consumercyclicals,  Consumernon-cyclicals,  Energy,  Industrials,  Realestate,  Technology,  Telecommunicationservices,  Utilities,  Variety
Variety
Cash,  Variety
tradable
Growth,  Acceleration
Types_cash,  Types_variety
Market_premium,  Smb,  Value,  Leverage,  Profitability,  Momentum


In [9]:
# ElasticNet regression
results_df = run_elastic_net(
    factors_df=distilled_factors,
    pct_changes=pct_changes,
    risk_free_df=risk_free_df,
    training_cutoff=training_cutoff,
    alphas=ENET_ALPHAS,
    l1_ratio=ENET_L1_RATIOS,
    cv=ENET_CV,
    tol=ENET_TOL,
)

Elastic Net Regression:   0%|          | 0/1538 [00:00<?, ?it/s]

In [10]:
# Post-regression factor screening
from scipy.stats import skew
from scipy.optimize import minimize_scalar

beta_cols = [col for col in results_df if col.endswith('beta')]
screening_df = pd.DataFrame(index=results_df.index)

screening_df['r2_adj_test'] = results_df['r2_test'].max() - results_df['r2_test']
screening_df['r2_adj_test'] = np.log1p(screening_df['r2_adj_test'] * optimize_scalar(screening_df['r2_adj_test']))
screening_df['r2_adj_test'] = screening_df['r2_adj_test'].max() - screening_df['r2_adj_test']

screening_df['cv_mse_std'] = results_df[['cv_mse_best','cv_mse_average','cv_mse_worst']].std(axis=1)
screening_df['cv_mse_std'] = np.log1p(screening_df['cv_mse_std'] * optimize_scalar(screening_df['cv_mse_std']))

screening_df['screening_score'] = screening_df['r2_adj_test'] / (1 + screening_df['cv_mse_std'])


display_df = pd.DataFrame({
    'mean_beta':  results_df[beta_cols].abs().mean(),
    'mean_adj_beta': results_df[beta_cols].abs().multiply(screening_df['screening_score'], axis=0).mean(),
    'non_zero_percentage': (results_df[beta_cols].abs() > 1e-6).sum() / len(results_df),
})
display_df.sort_values(by='non_zero_percentage')

11.100525636130623
140779340.18914315


,mean_beta,mean_adj_beta,non_zero_percentage
top10_variety_beta,0.653550,0.179288,0.807542
factor_profitability_beta,0.794617,0.223275,0.853056
industries_Technology_beta,0.361283,0.103690,0.864109
industries_Industrials_beta,0.350284,0.093210,0.871261
factor_momentum_beta,0.422751,0.124261,0.873862
gdp_pcap_value_beta,0.507127,0.129061,0.881664
factor_value_beta,0.644756,0.168718,0.884915
industries_variety_beta,0.370173,0.100259,0.896619
population_growth_beta,0.309332,0.084192,0.903771
industries_ConsumerCyclicals_beta,0.430272,0.125164,0.907672


In [11]:
# Factor-based ER
beta_cols = [col for col in results_df.columns if col.endswith('beta')]
asset_betas = results_df[beta_cols]
asset_betas.columns = [col.replace('_beta', '') for col in beta_cols]
asset_betas = asset_betas[distilled_factors.columns]

factor_premia = distilled_factors.mean()
# factor_premia[factor_premia > 0] = 0
# factor_premia *= -1

systematic_returns = asset_betas.dot(factor_premia)
factor_based_er = results_df['jensens_alpha'] + systematic_returns

screening_df = pd.DataFrame(index=results_df.index)
screening_df['expected_return'] = factor_based_er
screening_df['expected_return'] -= screening_df['expected_return'].min()

screening_df['r2_test'] = results_df['r2_test'].max() - results_df['r2_test']
screening_df['r2_test'] = np.log1p(screening_df['r2_test'] * optimize_scalar(screening_df['r2_test']))
screening_df['r2_test'] = screening_df['r2_test'].max() - screening_df['r2_test']

screening_df['cv_mse_std'] = results_df[['cv_mse_best','cv_mse_average','cv_mse_worst']].std(axis=1)
screening_df['cv_mse_std'] = np.log1p(screening_df['cv_mse_std'] * optimize_scalar(screening_df['cv_mse_std']))
screening_df['cv_mse_std'] = screening_df['cv_mse_std'].max() - screening_df['cv_mse_std']

scaler = MinMaxScaler()
screening_df[['r2_test', 'cv_mse_std']] = scaler.fit_transform(screening_df[['r2_test', 'cv_mse_std']])
screening_df['r2_adjusted_er'] = screening_df['expected_return'] * screening_df['r2_test'] * screening_df['cv_mse_std']
screening_df['historical_er'] = pct_changes.mean()

mu_utility = screening_df['r2_adjusted_er'] 
mu_historical = pct_changes.mean()

11.100525636130623
140779340.18914315


In [12]:
# Factor-based COV
factor_cov_matrix = distilled_factors.cov()
idiosyncratic_variances = results_df['mse_train']
D = np.diag(results_df['mse_train'])

systematic_cov = asset_betas.values @ factor_cov_matrix.values @ asset_betas.values.T
S = pd.DataFrame(
    systematic_cov + D,
    index=results_df.index,
    columns=results_df.index
)

# Static Risk-free rate 
rf_rate = risk_free_df['daily_nominal_rate'].iloc[-10:].mean()

In [13]:
# Portfolio Mean-Variance + Factor Exposure Balance Optimization
import cvxpy as cp
from pypfopt import base_optimizer

def portfolio_factor_dispersion(w, asset_betas):
    portfolio_betas = w @ asset_betas
    n_factors = asset_betas.shape[1]
    demeaned_betas = portfolio_betas - (cp.sum(portfolio_betas) / n_factors)
    return cp.norm(demeaned_betas, 2)

def optimize_with_factor_balance(mu, S, asset_betas, lambda_dispersion, lambda_risk, solver='CLARABEL'):
    n_assets = len(mu)
    w = cp.Variable(n_assets)

    expected_return = mu.values @ w
    portfolio_risk = cp.quad_form(w, S)
    factor_dispersion = portfolio_factor_dispersion(w, asset_betas.values)
    
    objective = cp.Maximize(
        expected_return
        - factor_dispersion * lambda_dispersion
        - portfolio_risk * lambda_risk
    )
    
    constraints = [cp.sum(w) == 1, w >= 0, w <= 1]
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=solver)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Warning: Optimal solution not found. Status: {problem.status}")
        return None

    weights = pd.Series(w.value, index=mu.index)
    return weights

def print_portfolio_stats(weights, mu, S, asset_betas):
    df = weights[weights > 0].sort_values(ascending=False)
    for k,v in df.items():
        row = filtered_df[filtered_df['conId'] == k]
        symbol = row['symbol'].iloc[0]
        print(f'{symbol}: {round(v*100, 2)}%')

    rf_rate = risk_free_df['daily_nominal_rate'].iloc[-10:].mean()
    er, volatility, sharpe_ratio = base_optimizer.portfolio_performance(weights, mu, S, risk_free_rate=rf_rate)
    final_portfolio_betas = weights @ asset_betas

    print(f'num_etfs: {len(weights[weights > 0])}')
    print(f'beta_std: {final_portfolio_betas.std()}')
    print(f'er: {er}')
    print(f'volatility: {volatility}')
    print(f'sharpe: {sharpe_ratio}\n')



# Single optimization
lambda_risk = 1
lambda_dispersion = 1

weights = optimize_with_factor_balance(
    mu=mu_utility,
    S=S,
    asset_betas=asset_betas,
    lambda_risk=lambda_risk, 
    lambda_dispersion=lambda_dispersion, 
)

In [14]:
def optimize_with_cardinality(mu, S, asset_betas, lambda_dispersion, lambda_risk, upper_bounds, max_assets, solver='SCIP'):
    n_assets = len(mu)
    w = cp.Variable(n_assets)
    z = cp.Variable(n_assets, boolean=True)

    expected_return = mu.values @ w
    portfolio_risk = cp.quad_form(w, S)
    factor_dispersion = portfolio_factor_dispersion(w, asset_betas.values)
    
    objective = cp.Maximize(
        expected_return
        - factor_dispersion * lambda_dispersion
        - portfolio_risk * lambda_risk
    )
    
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        cp.sum(z) <= max_assets,
        w <= upper_bounds * z,
    ]
    
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=solver, verbose=True)
    # problem.solve(solver=solver)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Warning: Optimal solution not found. Status: {problem.status}")
        # return None

    weights = pd.Series(w.value, index=mu.index)
    return weights



lambda_risk = 1
lambda_dispersion = 1
upper_bounds = 1

first_weights = weights.sort_values(ascending=False).head(100)
mu_post = mu_utility[first_weights.index]
mu_historical_post = mu_historical[first_weights.index]
S_post = S.loc[mu_post.index, mu_post.index]
betas_post = asset_betas.loc[mu_post.index]

evaluated_results = []
test_range = range(7, 10)
for max_assets in tqdm(test_range, total = len(test_range)):
    start = datetime.now()
    weights = optimize_with_cardinality(
        mu=mu_post,
        S=S_post,
        asset_betas=betas_post,
        lambda_dispersion=lambda_dispersion,
        lambda_risk=lambda_risk,
        upper_bounds=upper_bounds,
        max_assets = max_assets,
    )
    print_portfolio_stats(weights, mu_post, S_post, betas_post)

    num_assets = len(weights[weights > 0])
    assert num_assets == max_assets

    er_model, std_model, sharpe_model = base_optimizer.portfolio_performance(weights, mu_post, S_post, risk_free_rate=rf_rate)
    er_hist, _, sharpe_hist = base_optimizer.portfolio_performance(weights, mu_historical_post, S_post, risk_free_rate=rf_rate)
    final_portfolio_betas = weights @ betas_post

    evaluated_results.append({
        'sharpe_model': sharpe_model,
        'sharpe_hist': sharpe_hist,
        'er_model': er_model,
        'er_hist': er_hist,
        'volatility': std_model,
        'factor_beta_std': final_portfolio_betas.std(),
        'num_assets': num_assets,
        'weights': weights[weights > 0],
        'duration': datetime.now() - start
    })


  0%|          | 0/3 [00:00<?, ?it/s]

(CVXPY) Jul 10 10:18:33 AM: Your problem has 200 variables, 202 constraints, and 0 parameters.
(CVXPY) Jul 10 10:18:33 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 10 10:18:33 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 10 10:18:33 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jul 10 10:18:33 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Jul 10 10:18:33 AM: Compiling problem (target solver=SCIP).
(CVXPY) Jul 10 10:18:33 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) Jul 10 10:18:33 AM: Applying reduction FlipObjective
(CVXPY) Jul 10 10:18:33 AM: Applying reduction Dcp2Cone
(CVXPY) Jul 10 10:18:33 AM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 10 10:18:33 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Jul 10 10:18:33 AM: Applying 

                                     CVXPY                                     
                                     v1.6.6                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
presolving:
(round 1, fast)       3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 

(CVXPY) Jul 10 11:32:45 AM: Problem status: optimal
(CVXPY) Jul 10 11:32:45 AM: Optimal value: -2.146e-02
(CVXPY) Jul 10 11:32:45 AM: Compilation took 1.117e-01 seconds
(CVXPY) Jul 10 11:32:45 AM: Solver (including time spent in interface) took 4.452e+03 seconds


-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------


...

(CVXPY) Jul 10 11:32:45 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 10 11:32:45 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 10 11:32:45 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jul 10 11:32:45 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Jul 10 11:32:45 AM: Compiling problem (target solver=SCIP).
(CVXPY) Jul 10 11:32:45 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) Jul 10 11:32:45 AM: Applying reduction FlipObjective
(CVXPY) Jul 10 11:32:45 AM: Applying reduction Dcp2Cone
(CVXPY) Jul 10 11:32:46 AM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 10 11:32:46 AM: Applying reduction ConeMatrixStuffing


-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------


(CVXPY) Jul 10 11:32:46 AM: Applying reduction SCIP
(CVXPY) Jul 10 11:32:46 AM: Finished problem compilation (took 2.907e-01 seconds).
(CVXPY) Jul 10 11:32:46 AM: Invoking solver SCIP  to obtain a solution.


-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
presolving:
(round 1, fast)       3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 3 del vars, 103 del conss, 0 add conss, 448 chg bounds, 0 chg sides, 0 chg coeffs, 101 upgd conss, 0 impls, 0 clqs
   (0.1s) probing cycle finished: starting next cycle
   (0.1s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.1s) no symmetry present (symcode time: 0.00)
presolving (4 rounds: 4 fast, 3 medium, 3 exhaustive):
 3 deleted vars, 103 deleted constraints, 0 added constraints, 448 tightened 

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(evaluated_results)#.to_csv('data/nphard.csv',index=False)

In [ ]:
# Old Bayesian optimization
# from skopt import gp_minimize
# from skopt.space import Real
# from skopt.utils import use_named_args
# from scipy.stats import norm

# def progress_callback(res):
#     pbar.update(1)

# midpoint = 0.05
# space  = [
#     # Real(0, midpoint, prior='uniform', name='cuttoff_threshold'),
#     # Real(midpoint+1e-15, .8, prior='uniform', name='upper_bounds'),
#     Real(1e-9, 1, prior='log-uniform', name='lambda_risk'),
#     Real(1e-9, 1, prior='log-uniform', name='lambda_dispersion'),
#     # Real(1e-5, 1e5, prior='log-uniform', name='lambda_l1'),
#     ]
# @use_named_args(space)
# def objective(**params):
#     weights = optimize_with_factor_balance(
#         mu=mu_utility,
#         S=S,
#         asset_betas=asset_betas,
#         # cuttoff_threshold=params['cuttoff_threshold'],
#         # upper_bounds=params['upper_bounds'],
#         lambda_risk=params['lambda_risk'],
#         lambda_dispersion=params['lambda_dispersion'],
#         # lambda_l1=params['lambda_l1'],
#     )
#     if weights.isna().sum():
#         score = 100
#         print(f'Empty: {round(params['cuttoff_threshold'], 3)} - {round(params['upper_bounds'], 3)}  Score: {score}')
#         return score

#     num_assets = len(weights[weights > 0])
#     final_portfolio_betas = weights @ asset_betas

#     er_model, std_model, sharpe_model = base_optimizer.portfolio_performance(weights, mu_utility, S, risk_free_rate=rf_rate)
#     er_hist, _, sharpe_hist = base_optimizer.portfolio_performance(weights, mu_historical, S, risk_free_rate=rf_rate)

#     # score = (sharpe_model - params['cuttoff_threshold']) / (num_assets + 4) # +4 to avoid large marginal changes for small num_assets
#     # score = (sharpe_model - params['cuttoff_threshold']) * (1 + normal_dist.pdf(num_assets)*5)
#     score = sharpe_model

#     evaluated_results.append({
#         # 'cuttoff_threshold': params['cuttoff_threshold'],
#         # 'upper_bounds': params['upper_bounds'],
#         'lambda_risk': params['lambda_risk'],
#         'lambda_dispersion': params['lambda_dispersion'],
#         # 'lambda_l1': params['lambda_l1'],
#         'score': score,
#         'sharpe_model': sharpe_model,
#         'sharpe_hist': sharpe_hist,
#         'er_model': er_model,
#         'er_hist': er_hist,
#         'volatility': std_model,
#         'factor_beta_std': final_portfolio_betas.std(),
#         'num_assets': num_assets,
#         'weights': weights[weights > 0],
#     })
#     return -score


# MEAN = 5
# STD = 2
# normal_dist = norm(loc=MEAN, scale=STD)

# evaluated_results = []
# n_calls = 50
# n_initial_points = n_calls//2
# n_initial_points = 2**4
# n_calls = n_initial_points # Delete later

# if 'pbar' in globals():
#     pbar.close()
# pbar = tqdm(total=n_calls)

# result = gp_minimize(
#     func=objective,
#     dimensions=space,
#     n_calls=n_calls,
#     # initial_point_generator='lhs',
#     initial_point_generator='sobol',
#     n_initial_points=n_initial_points,
#     random_state=42,
#     callback=[progress_callback],
# )

In [ ]:
# Plot parameter exploration 2d
import matplotlib.pyplot as plt

eval_df = pd.DataFrame(evaluated_results)

x = 'lambda_l1'
y = 'lambda_risk'

plt.scatter(eval_df[x], eval_df[y], c=eval_df['score'], cmap='viridis')
# plt.scatter(optimal_dict[x], optimal_dict[y], color='red', marker='D')
plt.xlabel(x)
plt.ylabel(y)
plt.colorbar()
plt.legend()
plt.show()

In [ ]:
# Plot 3D
import plotly.express as px

def to_text(df):
    lines = []
    for k,v in df.items():
        row = filtered_df[filtered_df['conId'] == k]
        symbol = row['symbol'].iloc[0]
        conid = int(row['conId'].iloc[0])
        lines.append(f'{conid} - {symbol}: {round(v*100, 2)}%')
    return '<br>' + '<br>'.join(lines)

eval_df = pd.DataFrame(evaluated_results)
eval_df['weights'] = eval_df['weights'].apply(lambda x: to_text(x.sort_values(ascending=False)))
eval_df['lambda_dispersion'] = np.log(eval_df['lambda_dispersion'])
eval_df['lambda_risk'] = np.log(eval_df['lambda_risk'])
# eval_df['lambda_l1'] = np.log(eval_df['lambda_l1'])

fig = px.scatter_3d(
    eval_df,
    x='score',
    y='lambda_risk',
    z='lambda_dispersion',
    color='num_assets',
    # size='num_assets',
    color_continuous_scale=px.colors.sequential.Viridis,
    hover_data={
        'score': True,
        'num_assets': True,
        'sharpe_model': True,
        'factor_beta_std': True,
        # 'cuttoff_threshold': True,
        'lambda_risk': True,
        'lambda_dispersion': True,
        'weights': True,
        }
)
fig.update_layout(height=700)
fig.show()

In [ ]:
filtered_df[filtered_df['conId'] == 311447467]

In [ ]:
eval_df['factor_dispersion'].iloc[0].value

In [ ]:
# Create the heatmap
from matplotlib import pyplot as plt
import seaborn as sns

eval_df = pd.DataFrame(evaluated_results)
eval_corr = eval_df.drop(['weights'], axis=1).corr()
plt.figure(figsize=(8, 6))
sns.heatmap(eval_corr, annot=True, cmap='coolwarm', center=0)
plt.show()

In [ ]:
from pypfopt import DiscreteAllocation
latest_prices = ... # Get the most recent prices for your ETFs
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=25000)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print(f"Funds remaining: ${leftover:.2f}")